# Read GCN SourceCode

#### 阅读 (Tkipf)[https://github.com/tkipf/gcn] 源码的笔记


In [1]:
import tensorflow as tf
import numpy as np

## inits.py: 使用不同的初始化方法初始化型为shape的向量

## layer.py: 定义父类Layer，派生出两个子类Dense(全连接层)和GraphConv

三个辅助函数：

- layer_uid: {每一层的名字：出现次数}的字典,get_layer_uid拿到该层出现的次数。
- dot(): wrapper of matmul，自己实现稀疏乘法
- dropout(): wrapper of dropout,稀疏矩阵的自己写，正常的调用tf.nn.dropput


**补充1： 稀疏张量**

tensorflow引用的sparse tensor的概念：因为图卷积输入的图表示矩阵（如邻接矩阵）是二维、稀疏的，所以要变成sparse表示。其实度矩阵也是sparse的（对角）。 SparseTensor创建稀疏张量，sparse_to_dense转换回稀疏矩阵。


indicies: 非零元素的索引

values: 非零元素值

dense_shape: 稀疏矩阵本来的型

sparse的乘法调用专门的tf函数进行。

In [14]:
ind = np.hstack([np.arange(1000).reshape(-1,1),np.arange(1000).reshape(-1,1)]) #(1000,2)
v   = np.ones(1000)*np.random.randint(10,100,1000)
d_shape = [1000,1000]
a = tf.SparseTensor(indices=ind, values=v, dense_shape=d_shape) #对角矩阵的稀疏表示
b = tf.sparse_to_dense(sparse_indices=ind,sparse_values=v,output_shape=d_shape,name='diagD')
with tf.Session() as sess:
    r = sess.run(b)
    print(r)

[[36.  0.  0. ...  0.  0.  0.]
 [ 0. 27.  0. ...  0.  0.  0.]
 [ 0.  0. 58. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 11.  0.  0.]
 [ 0.  0.  0. ...  0. 26.  0.]
 [ 0.  0.  0. ...  0.  0. 29.]]


**补充2：dropout:**

正则化的一种方法，每次训练时随机使每一层的一批神经元“失活”，来达到效果。

比如：输入是X，神经元是A

X = $\left(\begin{array}{ccc}         
           x^{i}_{1} \\
           . \\
           . \\
           . \\
           x^{i}_{n}
           \end{array} \right)$          
A = $\left(\begin{array}{ccc}         
           a^{l}_{1} \\
           . \\
           . \\
           . \\
           a^{l}_{n}
           \end{array} \right)$ 
那么在dense layer 输出就是X*A = $\sum_{j=1}^{n}x^{i}_{j}a^{l}_{j}$，dropout就是随机失活A的分量，所以输出的A*X含有的成分每次都在变化。

其他的一些Intuition:
1. dropout掉不同的隐藏神经元就类似在训练不同的网络，随机删掉一半隐藏神经元导致网络结构已经不同，整个dropout过程就相当于对很多个不同的神经网络取平均。而不同的网络产生不同的过拟合，一些互为“反向”的拟合相互抵消就可以达到整体上减少过拟合。
2. dropout程序导致两个神经元不一定每次都在一个dropout网络中出现。这样权值的更新不再依赖于有固定关系的隐含节点的共同作用，阻止了某些特征仅仅在其它特定特征下才有效果的情况 。迫使网络去学习更加鲁棒的特征 ，这些特征在其它的神经元的随机子集中也存在。
3. 为什么要把dropout结果进行缩放：比如一个神经元的输出是x，那么在训练的时候它有p的概率参与训练，(1-p)的概率丢弃，那么它输出的期望是px+(1-p)0=px。所以要把输出结果px逆向乘以1/p得到x。

**代码实现逻辑：**

layer.py提供网络的骨架，没有动态的数据流过程。

Layer定义了：__init__   __call__   _call   _log_vars_这些基本方法。其中构造函数主要作用是
构造该层的名字和决定是否记录，并初始化内部成员。这些内部成员是空的或者是初始值，留给子类自己去实现。__call__把该类变成可调用的了，就是可以把input传参进来，它会调用真正的计算函数_call，由_call完成不同层的计算（全连接 or 卷积）。子类自动继承了__call__方法，所以不需要再重写，只需要实现自己的_call就行。

自己实现时可以先舍弃掉所有的logging，之后再补上。

重要的内部成员有：<b>name, logging, var</b>, 其他都是通过构造函数传参而形成的。

每层名字的格式： dense/graphconvolution_ith(第几个XX层)。

Dense继承Layer，包含两个方法__init__ 和 _call 。 自己的构造器首先调用Layer的构造器实现本层的name和logging，然后定义一些内部成员，通过传参赋值，然后进行变量的初始化。想想一个层需要哪些参数？就能理解为什么那么多参数传进来了。

进而实现_call: 首先dropout然后dot。注意稀疏性和bias。

GraphConv继承Layer，其他都与Dense相同，只不过在_call里的实现是图卷积操作。

- 注意：

placeholder是每层传进来的参数，dropout, support都是通过它传进来的。为什么要用placeholder?因为这些数据是需要在动态的数据流里feed给每次训练or每层的，与权重参数不同，权重参数是网络骨架的一部分。


作用域。每层初始化参数时都要用Variable_scope()放在该范围里，可以共享。然后在Layer那里的__call__方法，输出的计算又是放在tf.name_scope(每层的名字)下的。 同一name_scope下的tf.Variable()同名变量会被自动进行设置别名，不同name_scope下的tf.Variable()同名变量，其完整变量名不同（因为name_scope不同），故它们不是同一变量。<b>所以组织时每一层都是一个namescope，这样你堆两个FC时虽然两层权重变量w但由于namescope不同所以最终的权重变量名字也不同。然后每一层的权重放到variable_scope里，留作共享。</b>

https://www.zhihu.com/question/54513728 here is a discussion about namescape 



Q: 关于support的解释：
support 是A+I的对称正则化矩阵。但是由于A是稀疏的，所以正则化矩阵也是稀疏的。这里不用矩阵表示而是改用tuple的list:

[([x1,y1],value,type),([x2,y2],value,type),...]

所以权重也没法用一个整的矩阵来表示，而是拆成小份w1,w2,..,wn，都有相同的维度[in,out]，n就是非零元素的个数。

所以矩阵乘法XW被拆成了一个n-loop:

每次计算稀疏矩阵乘法X*wi，然后和A+I的稀疏表示第i个元素做乘法。

loop结束时把它们加和起来就得到了本层结果。




## models.py：组装网络骨骼，形成MLP&GCN,并模拟跑整个网络（假装有input而且执行所有计算步骤）

加入input，调用layer进行计算，并输出loss, accuracy, pred等结果。

先写父类model，定义了基本的方法和成员，然后子类MLP和GCN继承父类model并利用layer搭建自己的骨架。

**model类的基本方法**： __init__    build   _build    predict   _loss   _accuracy   save/load

__init__: 和layer那里的一样，先拿到模型的名字(mlp or graphconvolution)和logging,然后初始化一些成员。这些成员需要子类或者方法去重新赋值，但是在init里面列出来了，有助于形成全局观，并且之后可以随时取用。根据model的流程，需要的成员有：

input/output   var   placeholder   一开始就需要

layers/activations 前传过程中需要

loss/accuracy/optimizer/opt_op 优化时需要


_build: 需要子类重新实现的函数，就是搭layers

build: 整个网络的run。首先调用_build把网络骨骼组织好，然后执行前传过程。前传就是把layer当成可调用函数并喂入数据流。之后把网络参数存储到var里，最后计算loss and accuracy并执行优化。这里对一些成员变量如output，opt_op，var ,layers,activations重新赋值。

_loss/_accuracy: 子类自己定义，给loss, accuracy赋值

_predict: 子类自己定义

save/load: 由model定义好。


**MLP/GCN类：** 拥有自己的__init__   , build   ,  _loss   ,  _accuracy   ,  _predict方法

__init__: 调用父类的构造器，并且初始化自己的input,input_dim, output_dim, placeholder, optimizer。最重要的是调用build方法进行整个网络的run。

_build: 组织自己的网络骨骼

_loss/_accuracy/_predict: 根据问题需要和网络自身写不同的函数
<br>
<br>
<br>
- 注：
placeholder在这里的作用：提供input"features", true_output"label" 以及计算准确率时的"label_mask"。

## train.py: 取到各种数据，给模型喂入数据并且计算，训练，更新